In [1]:
# === T0: 공통 유틸 ===
import os, cv2, numpy as np, glob, pathlib, time

CAL_PATH = "calib_out/calib.npz"  # 방금 만든 npz
cal = np.load(CAL_PATH, allow_pickle=True)
MODEL = str(cal["model"])
K, D = cal["K"], cal["D"]
newK = cal["newK"]
MAP1, MAP2 = cal["map1"], cal["map2"]
W, H = map(int, cal["img_size"])
SIZE = (W, H)

def undistort_img_bgr(bgr):
    """npz의 map1/2로 이미지 보정"""
    h,w = bgr.shape[:2]
    # 크기가 npz의 맵과 다르면 새 맵 생성
    if (w,h) != (W,H):
        if MODEL=="pinhole":
            nK, _ = cv2.getOptimalNewCameraMatrix(K,D,(w,h),0,(w,h))
            m1,m2 = cv2.initUndistortRectifyMap(K,D,None,nK,(w,h),cv2.CV_16SC2)
        else:
            R = np.eye(3); nK = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(K,D,(w,h),R,balance=0.0)
            m1,m2 = cv2.fisheye.initUndistortRectifyMap(K,D,R,nK,(w,h),cv2.CV_16SC2)
        return cv2.remap(bgr, m1, m2, cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
    else:
        return cv2.remap(bgr, MAP1, MAP2, cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

def side_by_side(bgr, ud, max_w=1800):
    """원본/보정 좌우 비교 이미지"""
    h = min(bgr.shape[0], ud.shape[0])
    bgr2 = cv2.resize(bgr, (int(bgr.shape[1]*h/bgr.shape[0]), h))
    ud2  = cv2.resize(ud,  (int(ud.shape[1]*h/ud.shape[0]), h))
    cat  = np.hstack([bgr2, ud2])
    if cat.shape[1] > max_w:
        cat = cv2.resize(cat, (max_w, int(max_w * cat.shape[0]/cat.shape[1])))
    return cat


In [6]:
# === T1: 테스트 폴더 일괄 보정 + 비교 JPG 저장 ===
IN_DIR  = "test_0"  # 테스트 이미지 폴더 (네 폴더로 바꿔)
OUT_DIR = pathlib.Path(IN_DIR + "_ud")
OUT_DIR.mkdir(parents=True, exist_ok=True)

paths = sorted(glob.glob(os.path.join(IN_DIR,"*.jpg")) + glob.glob(os.path.join(IN_DIR,"*.png")))
print("found", len(paths), "images")

for p in paths:
    img = cv2.imread(p, cv2.IMREAD_COLOR)
    if img is None: 
        print("read fail", p); 
        continue
    und = undistort_img_bgr(img)
    cmp = side_by_side(img, und)
    cv2.imwrite(str(OUT_DIR / (pathlib.Path(p).stem + "_cmp.jpg")), cmp)
    cv2.imwrite(str(OUT_DIR / (pathlib.Path(p).stem + "_ud.jpg")), und)
print("done →", OUT_DIR.resolve())


found 7 images
done → C:\Users\gmlwn\OneDrive\바탕 화면\ICon1학년\광통신\PTCamera_waveshare\test_set\chess_try_1\test_0_ud


In [3]:
# === T2: 좌/우 가장자리 직선성 점수 (낮을수록 좋음) ===
def straightness_score(bgr, side="L", band=0.10):
    h,w = bgr.shape[:2]
    x0,x1 = (0, int(w*band)) if side=="L" else (int(w*(1-band)), w)
    roi = bgr[:, x0:x1]
    g = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    e = cv2.Canny(g, 60, 180)
    ys, xs = np.where(e>0)
    if len(xs)<200: 
        return None  # 에지가 부족
    # ROI 좌표를 원본 좌표로
    xs = xs + x0
    pts = np.column_stack([xs, ys]).astype(np.float32)

    # x = a*y + b 로 최소제곱 직선 적합 후 수직 잔차(RMSE) 계산
    A = np.vstack([ys, np.ones_like(ys)]).T
    a,b = np.linalg.lstsq(A, xs, rcond=None)[0]
    xs_fit = a*ys + b
    rmse = float(np.sqrt(np.mean((xs - xs_fit)**2)))
    return rmse

def print_scores(img):
    sL = straightness_score(img, "L")
    sR = straightness_score(img, "R")
    return sL, sR

test = paths[:8] if 'paths' in globals() and len(paths)>0 else []
for p in test:
    img = cv2.imread(p)
    und = undistort_img_bgr(img)
    sL0,sR0 = print_scores(img)
    sL1,sR1 = print_scores(und)
    print(pathlib.Path(p).name, " Straightness RMSE(px)  ",
          f"L: {sL0:.2f}->{sL1:.2f}" if sL0 and sL1 else "L: n/a",
          f" R: {sR0:.2f}->{sR1:.2f}" if sR0 and sR1 else "R: n/a")


img_t+00_p+000_20250904_200150_911.jpg  Straightness RMSE(px)   L: 70.20->72.28  R: 76.60->78.49
img_t+00_p+000_20250904_200354_489.jpg  Straightness RMSE(px)   L: 76.61->69.80  R: 70.59->74.88
img_t+00_p+030_20250904_200149_743.jpg  Straightness RMSE(px)   L: 74.10->73.43  R: 66.03->77.30
img_t+00_p+030_20250904_200352_379.jpg  Straightness RMSE(px)   L: 58.64->73.23  R: 82.09->65.90
img_t+00_p+060_20250904_200148_656.jpg  Straightness RMSE(px)   L: 75.71->72.06  R: 62.24->73.75
img_t+00_p+060_20250904_200350_257.jpg  Straightness RMSE(px)   L: 67.99->66.14  R: 69.51->73.41
img_t+00_p+090_20250904_200147_568.jpg  Straightness RMSE(px)   L: 78.23->71.96  R: 82.91->60.62
img_t+00_p+090_20250904_200348_094.jpg  Straightness RMSE(px)   L: 73.27->73.84  R: 88.80->69.43
